In [10]:
#Import lIBRARIES
import pandas as pd
import numpy as np
import os as os

In [11]:
# example making new folder with os

os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['Final_basics.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2001.json']

In [12]:
##Load Basics set and assign to DF

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [13]:
#Assign local file to DF
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [14]:
#Basics preprocessing

#Replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True) 
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [15]:
#Eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['genres', 'runtimeMinutes'])

In [16]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           35687
endYear           2313293
runtimeMinutes          0
genres                  0
dtype: int64

In [17]:
#keep only titleType==Movie
movie_filter = basics['titleType']=='movie'

basics[movie_filter]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1172,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"
1273,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"
...,...,...,...,...,...,...,...,...,...
8998840,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"
8998924,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,0,2019,NaN,123,Drama
8998965,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
8998992,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,NaN,100,Documentary


In [18]:
#keep startYear 2000-2022
start_filt = basics['startYear'] >= '2000'
start_filt.head()

0    False
1    False
2    False
3    False
4    False
Name: startYear, dtype: bool

In [19]:
end_filt = basics['startYear']<='2022'
end_filt.head()

0    True
1    True
2    True
3    True
4    True
Name: startYear, dtype: bool

In [20]:
basics = basics.loc[start_filt & end_filt, :]
basics.info

<bound method DataFrame.info of             tconst  titleType                    primaryTitle  \
13082    tt0013274      movie     Istoriya grazhdanskoy voyny   
33805    tt0034413      short              Youth Gets a Break   
34805    tt0035423      movie                  Kate & Leopold   
39547    tt0040241      short                 Color Rhapsodie   
43551    tt0044326      short                       Abstronic   
...            ...        ...                             ...   
8999025  tt9916754      movie  Chico Albuquerque - Revelações   
8999031  tt9916766  tvEpisode                  Episode #10.15   
8999066  tt9916840  tvEpisode      Horrid Henry's Comic Caper   
8999073  tt9916856      short                        The Wind   
8999074  tt9916880  tvEpisode       Horrid Henry Knows It All   

                          originalTitle isAdult startYear endYear  \
13082       Istoriya grazhdanskoy voyny       0      2021     NaN   
33805                Youth Gets a Break       0  

In [21]:
#Eliminate movies that include  "Documentary" in genre (see tip below)
#basics = basics[basics["genres"].str.contains("Documentary") == False]
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1421564 entries, 33805 to 8999074
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1421564 non-null  object
 1   titleType       1421564 non-null  object
 2   primaryTitle    1421564 non-null  object
 3   originalTitle   1421564 non-null  object
 4   isAdult         1421564 non-null  object
 5   startYear       1421564 non-null  object
 6   endYear         21130 non-null    object
 7   runtimeMinutes  1421564 non-null  object
 8   genres          1421564 non-null  object
dtypes: object(9)
memory usage: 108.5+ MB


In [22]:
basics['genres'].value_counts()

Drama,Short                       97599
Drama                             89756
Comedy                            85599
Reality-TV                        73599
Short                             72748
                                  ...  
Action,Fantasy,War                    1
Family,Fantasy,News                   1
Animation,Game-Show,Reality-TV        1
Family,Short,Talk-Show                1
Action,History,Western                1
Name: genres, Length: 1766, dtype: int64

In [23]:
##Load AKAs data set and save locally 
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [24]:
#Assign local file to DF
akas = pd.read_csv("Data/title.akas.csv.gz", low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [25]:
akas.replace({'\\N':np.nan}, inplace=True)

In [26]:
#AKA - keep only US entries. I keep breaking the main df, I'm creating a dummy
#df that I will move back once done.
dfa= akas
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [27]:
#Apply filter to retain only movies in US region
region_filt = dfa['region']=='US'
region_filt.head()

0    False
1    False
2    False
3    False
4    False
Name: region, dtype: bool

In [28]:
#Check that filtering worked...it did.  ONLY US region films are showing.
dfa[region_filt].head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [29]:
dfa = dfa.loc[region_filt, :]
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [30]:
dfa['region'].value_counts()

US    1328708
Name: region, dtype: int64

In [31]:
#Transfer dummy set back to actual df
akas = dfa
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [32]:
#Save new cleansed df to local csv file from earlier 
akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [33]:
#Display df info as requested
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1328708 entries, 5 to 32328026
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1328708 non-null  object
 1   ordering         1328708 non-null  int64 
 2   title            1328708 non-null  object
 3   region           1328708 non-null  object
 4   language         3581 non-null     object
 5   types            1026140 non-null  object
 6   attributes       44187 non-null    object
 7   isOriginalTitle  1327333 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.2+ MB


In [34]:
##Load Ratings data set and assign to DF
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [35]:
#Assign local file to DF
ratings = pd.read_csv("Data/title.ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1888
1,tt0000002,5.9,251
2,tt0000003,6.5,1679
3,tt0000004,5.8,164
4,tt0000005,6.2,2498


In [36]:
#RATINGS Preprocessing
ratings = ratings.replace({'\\N':np.nan}) 

In [37]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254186 entries, 0 to 1254185
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1254186 non-null  object 
 1   averageRating  1254186 non-null  float64
 2   numVotes       1254186 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [38]:
ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [39]:
del ratings

In [40]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])


In [41]:
basics = basics[keepers]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 470954 entries, 33805 to 8999011
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          470954 non-null  object
 1   titleType       470954 non-null  object
 2   primaryTitle    470954 non-null  object
 3   originalTitle   470954 non-null  object
 4   isAdult         470954 non-null  object
 5   startYear       470954 non-null  object
 6   endYear         9535 non-null    object
 7   runtimeMinutes  470954 non-null  object
 8   genres          470954 non-null  object
dtypes: object(9)
memory usage: 35.9+ MB


In [42]:
basics.to_csv("Data/Final_basics.csv.gz", compression= 'gzip',index=False)
